In [1]:
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth
else:
    # Do this only in Colab notebooks! Otherwise use pip install unsloth
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft trl triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install --no-deps unsloth

In [2]:
import torch
print(torch.__version__)
print(torch.cuda.is_available())
print(torch.cuda.get_device_name(0))


2.6.0+cu124
True
Tesla P100-PCIE-16GB


In [3]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = True # Use 4bit quantization to reduce memory usage. Can be False.

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-1.7B-unsloth-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


2025-07-05 14:47:56.249049: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:477] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1751726876.489657      35 cuda_dnn.cc:8310] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1751726876.562962      35 cuda_blas.cc:1418] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.6.12: Fast Qwen3 patching. Transformers: 4.51.3.
   \\   /|    Tesla P100-PCIE-16GB. Num GPUs = 1. Max memory: 15.888 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 6.0. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/1.41G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/237 [00:00<?, ?B/s]

...

merges.txt: 0.00B [00:00, ?B/s]

added_tokens.json:   0%|          | 0.00/707 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/614 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

chat_template.jinja: 0.00B [00:00, ?B/s]

In [4]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 128, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",

                      "embed_tokens", "lm_head",], # Add for continual pretraining
    lora_alpha = 32,
    lora_dropout = 0, # Supports any, but = 0 is optimized
    bias = "none",    # Supports any, but = "none" is optimized
    # [NEW] "unsloth" uses 30% less VRAM, fits 2x larger batch sizes!
    use_gradient_checkpointing = "unsloth", # True or "unsloth" for very long context
    random_state = 3407,
    use_rslora = True,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth: Offloading input_embeddings to disk to save VRAM
Unsloth: Offloading output_embeddings to disk to save VRAM


Unsloth 2025.6.12 patched 28 layers with 28 QKV layers, 28 O layers and 28 MLP layers.


Unsloth: Training embed_tokens in mixed precision to save VRAM
Unsloth: Training lm_head in mixed precision to save VRAM


In [6]:
!gdown 1OnxJ_UeJ_YXRX0E1U7lBIxqI9phaI6wq

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)


Downloading...
From (original): https://drive.google.com/uc?id=1OnxJ_UeJ_YXRX0E1U7lBIxqI9phaI6wq
From (redirected): https://drive.google.com/uc?id=1OnxJ_UeJ_YXRX0E1U7lBIxqI9phaI6wq&confirm=t&uuid=8f7625fd-2973-45b4-bfaf-c7f81afe8a7c
To: /kaggle/working/legal_corpus.json
100%|████████████████████████████████████████| 117M/117M [00:01<00:00, 83.4MB/s]


In [11]:
import json

with open("legal_corpus.json", "r", encoding="utf-8") as f:
    data = json.load(f)

with open("formatted_dataset.jsonl", "w", encoding="utf-8") as out_file:
    for law in data:
        law_id = law.get("law_id", "unknown")
        for article in law.get("content", []):
            article_text = article.get("content_Article", "").strip()
            if article_text:
                item = {"text": f"{law_id}: {article_text}"}
                json.dump(item, out_file, ensure_ascii=False)
                out_file.write("\n")


In [13]:
with open("formatted_dataset.jsonl", "r", encoding="utf-8") as f:
    for i in range(1):
        print(f.readline())


{"text": "14/2022/TT-NHNN: 1. Thông tư này quy định mã số, tiêu chuẩn chuyên môn, nghiệp vụ và xếp lương đối với các ngạch công chức chuyên ngành Ngân hàng.\n\n2. Thông tư này áp dụng đối với công chức làm việc tại các đơn vị thuộc Ngân hàng Nhà nước Việt Nam (gọi tắt là Ngân hàng Nhà nước)."}



In [14]:
from datasets import load_dataset

dataset = load_dataset("json", data_files="formatted_dataset.jsonl", split="train")


Generating train split: 0 examples [00:00, ? examples/s]

In [17]:
from transformers import TrainingArguments
from unsloth import is_bfloat16_supported
from unsloth import UnslothTrainer, UnslothTrainingArguments

trainer = UnslothTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,

    args = UnslothTrainingArguments(
        per_device_train_batch_size = 4,
        gradient_accumulation_steps = 16,
    
        num_train_epochs = 1,
        learning_rate = 2e-5,
        embedding_learning_rate = 2e-6,
        warmup_ratio = 0.1,
        lr_scheduler_type = "linear",
        fp16 = not is_bfloat16_supported(),
        bf16 = is_bfloat16_supported(), 
        #bf16 = True,
        #fp16 = False,
        logging_steps = 10,
        optim = "adamw_8bit",
        weight_decay = 0.01,
        seed = 3407,
        output_dir = "outputs",
    ),
)


Unsloth: Tokenizing ["text"]:   0%|          | 0/59628 [00:00<?, ? examples/s]

In [ ]:
trainer_stats = trainer.train()
from huggingface_hub import login
import os

# Set HF_TOKEN environment variable before running
hf_token = os.getenv("HF_TOKEN")
if hf_token:
    login(token=hf_token)
    model.push_to_hub("thailevann/Qwen3-1.7B_CT_VLSP_track5") 
    tokenizer.push_to_hub("thailevann/Qwen3-1.7B_CT_VLSP_track5")
else:
    print("Warning: HF_TOKEN environment variable not set. Skipping model upload.")